In [ ]:
global sal
sal = pd.read_csv('sample.csv',encoding = 'ISO-8859-1')

ProcessDataset(dataset=None,Predict=None,Predictors=None,CreateNLPFeatureFrom=None)

In [ ]:
sal.head()

In [ ]:
DropColumn([])
#PrintDistribution(columnList=['gross_income'])

In [ ]:
rowCountList={}
ProcessDataset(Predict=None,Predictors=None,CreateNLPFeatureFrom=None)

In [ ]:
PrintDistribution(['customer_sex'])

In [ ]:
sal['']

In [ ]:
DropColumn(['X_DESCRIPTION','CREATED','NAME'])

In [ ]:
PrintDistribution(['ratings'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from textblob import TextBlob
import operator
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline



def printData():
    pd.DataFrame(rowCountList,index=[0]).iplot(kind='bar',title='Percentage Unique Values')
    sal.corr().iplot()

def calculcateNullCOlumns():
    if len(sal.isnull().sum().nonzero()) > 0:
        sal.isnull().sum().iplot(kind='bar',title='NULL Values')

def SeeDistributionOfLabel(Predict,Predictors):
    #this can be imporved 
    sorted_x = sorted(rowCountList.items(), key=operator.itemgetter(1))[-2:]
    if Predictors!= None:
        sal.groupby(Predict).count()[Predictors].iplot(kind='bar')
    else:
        sal.groupby(Predict).count()[[sorted_x[0][0],sorted_x[1][0]]].iplot(kind='bar')
        

def explainData():
    global rowCountList 
    for col in sal.columns:
        rowCount = sal[col].count()
        if rowCount!=0:
            rowCountList[col]=(len(set(sal[col].unique()))/rowCount)*100
        else:
            rowCountList[col]=0

            
def ProcessDataset(dataset=None,Predict=None,Predictors=None,CreateNLPFeatureFrom=None):
    sal.info()
    explainData()
    printData()
    calculcateNullCOlumns()
    if Predict != None:
        SeeDistributionOfLabel(Predict,Predictors)
    if CreateNLPFeatureFrom != None:
        sal['sentimentOf{}'.format(CreateNLPFeatureFrom)] = sal.apply (lambda row: BuildASentimentFeature (row[CreateNLPFeatureFrom]),axis=1)
        sns.countplot(x='sentimentOf{}'.format(CreateNLPFeatureFrom),data=sal)

        
def BuildASentimentFeature(text):
    text = TextBlob(text)
    if text.sentiment.polarity > 0:
        return "Positive"
    elif text.sentiment.polarity < 0:
        return "Negative"
    else:
        return "Neutral"

    
def DropColumn(columnList=None):
    if columnList != None:
        sal.drop(columnList,axis=1,inplace=True)
        
def PrintDistribution(columnList=None):
    if columnList != None:
        for column in columnList:
            a4_dims = (14.7, 10.27)
            fig, ax = plt.subplots(figsize=a4_dims)
            sns.distplot(sal[column].dropna(),ax=ax)
            print(sal[column].value_counts(dropna=True).head())
            

def BulkDropColumnsBeyoundAThresh(thresh=0):   
    for c in sal:
        if sum(sal[c].isnull()) >= thresh:
            DropColumn(c)

def EncodeObjectAttributes():
    from sklearn import preprocessing
    for ml in list(sal.select_dtypes(include=['object']).columns):
        le = preprocessing.LabelEncoder()
        le.fit(sal[ml])
        sal[ml] = le.transform(sal[ml])
        
def FillNullValueWithMode():
    sal[list(sal)]=sal[list(sal)].fillna(sal.mode().iloc[0])

In [ ]:
rowCountList={}
ProcessDataset(Predict=None,Predictors=None,CreateNLPFeatureFrom=None)

In [ ]:
Predict('Sex')


In [ ]:
Predict2('Parch')

In [ ]:
def Predict(Col=None):
    from sklearn.cross_validation import train_test_split
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import classification_report,confusion_matrix
    X = sal.drop(Col,axis=1)
    y = sal[Col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
    dtree = DecisionTreeClassifier()
    dtree.fit(X_train,y_train)
    predictions = dtree.predict(X_test)
    print(confusion_matrix(y_test,predictions))
    print('\n')
    print(classification_report(y_test,predictions))


In [ ]:
def Predict2(col=None):
    from sklearn.cross_validation import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report,confusion_matrix
    X = sal.drop(col,axis=1)
    y = sal[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
    rfc = RandomForestClassifier(n_estimators=100)
    rfc.fit(X_train,y_train)
    rfc_pred = rfc.predict(X_test)
    print(confusion_matrix(y_test,rfc_pred))
    print('\n')
    print(classification_report(y_test,rfc_pred))

In [ ]:
BulkDropColumnsBeyoundAThresh(thresh=17000)

In [ ]:
FillNullValueWithMode()

In [ ]:
EncodeObjectAttributes()